In [ ]:
import pandas as pd
import json

In [ ]:
with open('./code/iJO1366.json', 'r') as f:
    l = f.readlines()

In [ ]:
print len(l)

In [ ]:
l[0]

In [ ]:
def kgk10_ko(model):
    # Remove genes that aren't in kck10 (need to translate to GPR)
    # A19:
        # rna-19
    # KC6: 
        # A19 strain
        # speA (arginine decarboxylase) -  b2938
        # tnaA (tryptophanase) - b3708
        # sdaA and sdaB (serine deaminase) - b1814 and b2797
        # gshA (glutatamte-cysteine ligase) - b2688
        # tonA - (tonB is b1252)
        # endAmet
    # KGK10:
        # KC6 strain
        # Remove gor (GSSG) - b3500
        # Add hemagglutinin tag to trxB - b0888

In [ ]:
# Works IN PLACE
def replace_comparts(model):
    #Move all reactions and metabolites to cytosol
    cmpts = model.compartments
    non_cyt = cmpts - 'c'
    for rxn in model.reactions:
        for non_cyt_cmpt in non_cyt:
            rxn.compartments.remove(non_cyt_cmpt)
    
    for metab in model.metabolites:
        metab.compartment = 'c'

# Works IN PLACE
def remove_comparts(model):
    #Remove all non-cytosolic reactions and metabolites
    for rxn in model.reactions:
        if not 'c' in rxn.compartments:
            rxn.remove_from_model(remove_orphans=True)
            
    for metab in model.metabolites:
        if metab.compartment != 'c':
            metab.remove_from_model(destructive=False)

In [ ]:
def create_obj_fxn(model):
    #Only care about protein synthesis for CFPS?

In [ ]:
# N-butanol pathway
# Glucose + NAD + PO4 + Mg + ATP -> NADH + ADP + 2xPEP
# PEP + ADP -> ATP + Pyruvate
# Pyr + CoA + NAD -> CO2 + NADH + Acc_CoA
# 2x Acc_CoA -> acetoacyl-CoA (AtoB enzyme)
# acetoacyl-CoA + NADH -> 3-hydroxybutyryl-CoA + NAD (Hbd enzyme)
# 3-hydroxybutyryl-CoA -> crotonyl-CoA (Crt enzyme)
# crotonyl-CoA + NADH -> butyryl-CoA + NAD (Ter enzyme)
# butyryl-CoA + NADH -> butyraldehyde + NAD (AdhE enzyme)
# butyraldehyde + NADH -> 1-butanol + NAD (AdhE enzyme)


In [ ]:
# Start and end are metabolites
def find_path(start, end):
    explored = set([])
    found = False
    frontier = [start]
    path = {}
    while frontier:
        metab = frontier.pop()
        if metab in explored:
            continue
        print
        print "METABOLITE: ", metab
        print
        if metab == end:
            break
        explored.add(metab)

        for rxn in model_e.metabolites.get_by_id(metab.id).reactions:
            if metab in rxn.reactants:
                #print rxn
                for prod in rxn.products:
                    if not (prod in explored or prod in frontier):
                        path[prod] = metab
                        frontier.append(prod)
    return path
path = find_path(model_e.metabolites.glc__D_c, model_e.metabolites.pep_c)

cur = model_e.metabolites.pep_c
while cur != model_e.metabolites.glc__D_c:
    print cur.id
    cur = path[cur]

In [ ]:
def remove_exs(model):
    model_no_ex = model.copy()
    model_no_ex.remove_reactions(reactions=model_no_ex.exchanges)
    return model_no_ex
model_no_ex = remove_exs(model)

# KEGG

In [ ]:
import Bio.KEGG as bk
from Bio.KEGG import REST
path = REST.kegg_find('PATHWAY', 'butanol')
path.readlines()

In [ ]:
import bioservices
from bioservices.kegg import KEGG
k = KEGG()
k.organism = 'cac'
k.get('adhe1')
k.organism = 'eco'
k.list('pathway').split('\t')
butanol = k.get('path:eco00650', "kgml")
but_par = k.parse_kgml_pathway('path:eco00650')
but_par['relations'], but_par['entries']
compounds_str = k.list('compound')
compounds = compounds_str.strip().split('\n')
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
coa = 'acetoacetyl-coa'

for compound in compounds:
    cpd_id, name = compound.split('\t')
    score = similar(name.lower(), coa)
    if  score > 0.5:
        print score
        print cpd_id, name
        #break

In [ ]:
from Bio.KEGG import REST

pathways = REST.kegg_list("pathway", "eco").read()

# Filter all human pathways for repair pathways
but_pathways = []
for line in pathways.rstrip().split("\n"):
    entry, description = line.split("\t")
    if "Butanoate" in description:
        but_pathways.append(entry)
but_pathways
but_genes = []
for pathway in but_pathways:
    pathway_file = REST.kegg_get(pathway).read()  # query and read each pathway

    # iterate through each KEGG pathway file, keeping track of which section
    # of the file we're in, only read the gene in each pathway
    current_section = None
    for line in pathway_file.rstrip().split("\n"):
        print line
        section = line[:12].strip()  # section names are within 12 columns
        if not section == "":
            current_section = section

        if current_section == "GENE":
            gene_identifiers, gene_description = line[12:].split("; ")
            gene_id, gene_symbol = gene_identifiers.split()

            if not gene_symbol in but_genes:
                but_genes.append(gene_symbol)